In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import numpy as np
import os
import pickle
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Path to your downloaded CIFAR-10 dataset
data_dir = r'C:\Users\ahmed\Documents\Python Scripts\Image classification ResNet\cifar-10-batches'

In [ ]:
# Function to unpickle the dataset files
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Load all training batches
train_data = []
train_labels = []

for i in range(1, 6):
    batch_file = os.path.join(data_dir, f'data_batch_{i}')
    batch = unpickle(batch_file)
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']

# Convert training data to numpy arrays
train_data = np.vstack(train_data).reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)

# Load the test batch
test_batch = unpickle(os.path.join(data_dir, 'test_batch'))
test_data = test_batch[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
test_labels = np.array(test_batch[b'labels'])

print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")


In [ ]:
class CIFAR10Dataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        # Convert numpy array to PIL image for transformation
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Prepare the datasets
train_dataset = CIFAR10Dataset(train_data, train_labels, transform=transform)
test_dataset = CIFAR10Dataset(test_data, test_labels, transform=transform)

# Load data into DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Load pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Freeze all layers except the final one
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer with the number of classes in CIFAR-10
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # CIFAR-10 has 10 classes

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
# Training the model
def train_model(model, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluation mode

            running_loss = 0.0
            running_corrects = 0

            # Loop through the data
            loader = train_loader if phase == 'train' else test_loader
            for i, (inputs, labels) in enumerate(loader):
                inputs = inputs.to(device)
                
                # Convert labels to LongTensor
                labels = labels.to(device, dtype=torch.long)

                # Zero the gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize only in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Track the loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # Print loss and accuracy every 100 batches
                if phase == 'train' and i % 100 == 0:
                    print(f'Batch {i}/{len(loader)}, Loss: {loss.item():.4f}')

            epoch_loss = running_loss / len(loader.dataset)
            epoch_acc = running_corrects.double() / len(loader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# Train the model
model = train_model(model, criterion, optimizer, num_epochs=5)

In [ ]:
# Evaluate the model on test data
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the test images: {100 * correct / total:.2f}%')

evaluate_model(model)

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'resnet18_cifar10.pth')